# Asimetrična šifra RSA

In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

## Naloga 4

Implementirajte asimetrično šifro RSA, tako da podate implementacijo algoritmov za ustvarjanje ključev, šifriranja in dešifriranja. Pri tem si pomagajte z dokumentacijo knjižnice cryptography: [RSA @ cryptography](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/)

Specifikacije so naslednje:

Algoritem `gen(int)`
- opcijsko vzame velikost ključa (modula); privzeta vrednost naj bo 2048 bitov;
- eksponent e naj bo 65537
- vrne naj par `(pk, sk)`

In [2]:
def gen(key_size=2048):
    sk = rsa.generate_private_key(
        public_exponent=65537,
        key_size=key_size)
    pk = sk.public_key()
    return pk, sk

In [3]:
def test_gen():
    from cryptography.hazmat.backends.openssl.rsa import _RSAPublicKey, _RSAPrivateKey
    pk, sk = gen()
    assert type(pk) == _RSAPublicKey
    assert type(sk) == _RSAPrivateKey
    
test_gen()

Šifrirni algoritem vzame tri argumente:
- `pk` javni ključ kot ga vrne algoritem `gen()`
- `ct` čistopis v bajtih
- opcijsko `aad`, pridruženi podatki v bajtih

Za podlaganje uporabite algoritem `OAEP`, za `MGF` algoritem `MGF1` z `SHA256` in za zgoščevalno funkcijo prav tako `SHA256`.

Kot rezultat vrnite bajte tajnopisa.

In [4]:
def enc(pk, message: bytes, aad: bytes=None) -> bytes:
    return pk.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=aad))

Dešifrirani algoritem vzame tri argumente:
- `sk` zasebni ključ kot ga vrne algoritem gen(),
- `ct` tajnopis v bajtih,
- opcijsko `aad`, pridruženi podatki v bajtih.

Podobno kot pri šifriranju: za podlaganje uporabite algoritem `OAEP`, za `MGF` algoritem `MGF1` z `SHA256` in za zgoščevalno funkcijo prav tako `SHA256`.

Kot rezultat vrnite bajte čistopisa oz. napako, če je tajnopis neveljaven.

In [5]:
def dec(sk, ct: bytes, aad: bytes=None) -> bytes:
    return sk.decrypt(
        ct,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=aad))

In [6]:
def test_enc_dec():
    pk, sk = gen()
    
    assert dec(sk, enc(pk, b"hello")) == b"hello"
    assert dec(sk, enc(pk, b"hello", b"world"), b"world") == b"hello"
    try:
        dec(sk, enc(pk, b"hello", b"world"))
        raise AssertionError
    except ValueError:
        pass
    
test_enc_dec()